# Step #1 Azure OpenAI Service - Chat on private data using LangChain

Import note this tutorial is for langchain 0.2
The functions below might not work with other langchain versions

In [52]:
import os
import openai
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = os.getenv("AZURE_OPENAI_VERSION")
openai.api_base = os.getenv('OPENAI_API_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Step #2 Define an LLM Config and Invoke it
First, we will simply invoke the LLM with a basic prompt.


In [53]:
# Init LLM and embeddings model
llm = AzureChatOpenAI(deployment_name="gpt4o", 
                      temperature=0.5,
                      max_tokens=100, 
                      openai_api_version="2023-03-15-preview")

response = llm.invoke("Tell me a joke")
print(response.content)

Sure, here's a joke for you:

Why don't skeletons fight each other?

They don't have the guts!


# Step #3 Two Agents Interact with Each Other
The core idea of agents is to use a language model to choose a sequence of actions to take. 
In chains, a sequence of actions is hardcoded (in code). 
In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.
More information: https://python.langchain.com/v0.1/docs/modules/agents/

Next we will determine the behavior of our llms configs. This allows us to reuse them for different purposes.

In [62]:
# Define configuration with system instructions
config_writer = {
    "deployment_name": "gpt4o",
    "temperature": 0.6,
    "max_tokens": 300,
    "openai_api_version": "2023-03-15-preview",
    "instruction": "You are a creative writing assistant."
}

# Init LLM model
llm_writer = AzureChatOpenAI(deployment_name=config_writer["deployment_name"], 
                      temperature=config_writer["temperature"],
                      max_tokens=config_writer["max_tokens"], 
                      openai_api_version=config_writer["openai_api_version"])

# Add system instruction to the prompt
user_prompt = 'Write an application letter for a job as a software engineer.'
response = llm_writer.invoke(f'{config_writer["instruction"]} ' + user_prompt)
content = response.content
print(content)
config_reviewer = {
    "deployment_name": "gpt4o",
    "temperature": 0.4,
    "max_tokens": 500,
    "openai_api_version": "2023-03-15-preview",
    "instruction": "You are a critical reviewer, who's an expert in reviewing others content and providing helpful advice for improvements." 
}

# Init LLM model
llm_params = {key: config_reviewer[key] for key in config_reviewer if key != "instruction"}  # Exclude 'instruction'
llm_reviewer = AzureChatOpenAI(**llm_params)

user_prompt = 'Review the following content: ' + content
response = llm_reviewer.invoke(f'{config_reviewer["instruction"]} ' + user_prompt)
content = response.content
print(content)